Generate Mock Dataset

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

# Define the number of rows in the dataset
num_rows = 1000

# Generate random dates
start_date = pd.to_datetime('2022-01-01')
end_date = pd.to_datetime('2022-12-31')
date_field = pd.date_range(start_date, end_date, periods=num_rows)


# Generate random daily sales values
#daily_sales = np.random.randint(1, 10001, size=num_rows)

# Define the state codes
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Generate random state codes
state_code = np.random.choice(state_codes, size=num_rows)

# Generate random integer fields
int_field1 = np.random.randint(1, 10001, size=num_rows)
int_field2 = np.random.randint(1, 10001, size=num_rows)
int_field3 = np.random.randint(1, 10001, size=num_rows)
int_field4 = np.random.randint(1, 10001, size=num_rows)
int_field5 = np.random.randint(1, 10001, size=num_rows)

# Create a DataFrame with the generated data
data = pd.DataFrame({
#    'Daily Sales': daily_sales,
    'StateCode': state_code,
    'var1': int_field1,
    'var2': int_field2,
    'var3': int_field3,
    'var4': int_field4,
    'var5': int_field5,
    'Date': date_field
})


data['Sales'] = 1000 + 1.2 * data['var1'] + 3.5*data['var2'] + 0.67* data['var3'] + 2.45*data['var4'] + data['var5']
data['Date'] = data['Date'].dt.strftime('%m/%d/%Y')

In [3]:
data.head()

,StateCode,var1,var2,var3,var4,var5,Date,Sales
0,VT,12,3619,1462,2950,189,01/01/2022,22076.94
1,NM,102,3143,3179,1397,7360,01/01/2022,25035.48
2,CO,7945,6008,1616,6928,5238,01/01/2022,54856.32
3,MN,117,7013,3142,6215,8680,01/02/2022,51697.79
4,IA,8492,1487,9565,4751,35,01/02/2022,34478.40


Model Against Sales

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Create the regression model
reg_model = LinearRegression()

# Define the independent variables
X = data[['var1', 'var2', 'var3']]

# Standardize var1, var2, and var3
scaler = StandardScaler()
X[['var1s', 'var2s', 'var3s']] = scaler.fit_transform(X[['var1', 'var2', 'var3']])
X1 = X[['var1s', 'var2s', 'var3s']]

# Define the dependent variable
y = data['Sales']

# Fit the model
results = reg_model.fit(X1, y)

/var/folders/2y/295w_nh94yg378k6m37nf1x00000gn/T/ipykernel_26190/1613018345.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['var1s', 'var2s', 'var3s']] = scaler.fit_transform(X[['var1', 'var2', 'var3']])
/var/folders/2y/295w_nh94yg378k6m37nf1x00000gn/T/ipykernel_26190/1613018345.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['var1s', 'var2s', 'var3s']] = scaler.fit_transform(X[['var1', 'var2', 'var3']])
/var/folders/2y/295w_nh94yg378k6m37nf1x00000gn/T/ipykernel_26190/1613018345.py:12: Setti

In [5]:
# Predict the sales using the regression model
predicted_sales = reg_model.predict(X1)

# Create a new DataFrame to store the predicted sales
predicted_data = data.copy()
predicted_data['Predicted Sales'] = predicted_sales

# Display the predicted sales
predicted_data[['StateCode', 'var1', 'var2', 'var3', 'Sales', 'Predicted Sales']]

,StateCode,var1,var2,var3,Sales,Predicted Sales
0,VT,12,3619,1462,22076.94,31622.809274
1,NM,102,3143,3179,25035.48,31124.540976
2,CO,7945,6008,1616,54856.32,50154.391134
3,MN,117,7013,3142,51697.79,45067.527307
4,IA,8492,1487,9565,34478.40,39656.918909
...,...,...,...,...,...,...
995,MI,8038,638,8081,42827.47,35079.023828
996,WA,8829,3089,6923,40873.36,44146.407334
997,FL,790,2571,143,37771.26,27959.310619
998,LA,1154,6917,5175,32366.65,47314.880678


In [6]:
results.intercept_, results.coef_

(45396.51059, array([ 3590.98058686, 10455.79378439,  1852.41853796]))

In [7]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# Calculate MAPE
mape = mean_absolute_percentage_error(data['Sales'], predicted_data['Predicted Sales'])

# Calculate R-squared
r_squared = r2_score(data['Sales'], predicted_data['Predicted Sales'])

mape, r_squared

(0.16200492888928225, 0.6938637459958497)